<a href="https://colab.research.google.com/github/nisanuro/CNG483-Project1/blob/master/CNG483_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [553]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_images(path):
    images = []

    for filename in os.listdir(path):
        img=cv2.imread(os.path.join(image_path, filename),cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)

    return images

In [0]:
def histogram(image, bin):
    bins = []
    width = int(256/bin)

    for low in range(0, bin * width, width):
        bins.append((low, low+width))

    hist = np.zeros(bin).astype('int32')

    for value in image:
        for i in range(0, len(bins)):
            if (bins[i][0] <= value < bins[i][1]):
                hist[i] += 1
    
    return hist


In [0]:
def color_histogram(img, bin):
    chans = cv2.split(img)
    colors = ("b", "g", "r")

    features = []
   
    for (chan, color) in zip(chans, colors):
        if color == 'b':
            bf = histogram(chan, bin)
        elif color == 'g':
            gf = histogram(chan, bin)
        else:
            rf = histogram(chan, bin)

    # combination of blue and red channel bins
    temp = np.transpose([np.tile(bf, len(gf)), np.repeat(gf, len(bf))])
    
    blue_green_hist = []    
    for i in temp:
        blue_green_hist.append(sum(i))

    # same operation with red channel bins
    temp = np.transpose([np.tile(rf, len(blue_green_hist)), np.repeat(blue_green_hist, len(rf))])

    color_hist = []    
    for i in temp:
        color_hist.append(sum(i))
    
    return color_hist

In [0]:
def split_image_into_grids(image, grid):
    print("split grids")
    
    grids = []

    height=image.shape[0]
    width=image.shape[1]

    M = height//grid
    N = width//grid

    for y in range(0, height, M):
        for x in range(0, width, N):
            y1 = y + M
            x1 = x + N
            grids.append(image[y:y+M, x:x+N])


    #cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0))

    #os.chdir("/content/drive/My Drive/SaveDeneme")
    #cv2.imwrite("im" + str(i) + '_' + str(x) + '_' + str(y)+".png",tiles)
    '''
    plt.figure()
    plt.imshow(grid) 
    plt.show()''' 

    return grids


        
     

In [0]:
def create_dataset(path):
    print("create dataset")
    class_paths = []
    images = []
    labels = []

    with os.scandir(path) as itr: 
        for subdir in itr : 
            if subdir.is_dir():
                p = path + '/' + subdir.name
                class_paths.append(p)
    
    for p in class_paths:
        for filename in os.listdir(p):
            if filename.endswith(".jpg"):
                labels.append(os.path.basename(os.path.normpath(p)))                
                img = cv2.imread(os.path.join(p, filename))
                if img is not None:
                    images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
   
    return images, labels


In [0]:
def feature_extraction(images, grid, bin, isRGB):
    print("feature extraction")
    features = []

    for i in images:
        i = cv2.resize(i, (256, 256))
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)

        grids = split_image_into_grids(i, grid)

        grid_hists = []

        for i in grids:
            if(isRGB):
                grid_hists.append(color_histogram(i, bin))
            else:
                grid_hists.append(histogram(i, bin))
        
        # Concatenation
        grids_flat = np.array(grid_hists).flatten()
        
        '''
        # Sum
        grids_flat = np.array(grid_hists)
        grids_flat = np.sum(grids_flat, axis=0)
        '''
            
        features.append(grids_flat)
        
    return features


In [0]:
def classification(trainI, trainL, testI, testL, k):
    print("classification")
    model = KNeighborsClassifier(n_neighbors=k, metric='euclidean', algorithm='brute', n_jobs=-1)
    model.fit(trainI, trainL)
    acc = model.score(testI, testL)
    print(acc)

In [562]:
if __name__ == "__main__":

    test_path = "/content/drive/My Drive/CNG483-Project 1/TestSet"
    train_path = "/content/drive/My Drive/CNG483-Project 1/TrainingSet"
    validation_path = "/content/drive/My Drive/CNG483-Project 1/ValidationSet"

    
    # testI, testL = create_dataset(test_path)
    trainI, trainL = create_dataset(train_path)
    validationI, validationL = create_dataset(validation_path)
    
    # grid --> 1, 2, 4
    # bins --> 1, 128, 256
    # k --> 1, 5, 10
    # isRGB --> True, False

    grid = 4
    bins = 128
    isRGB = True
    k = 5
    
    trainI_f = feature_extraction(trainI, grid, bins, isRGB)
    validationI_f = feature_extraction(validationI, grid, bins, isRGB)

    classification(trainI_f, trainL, validationI_f, testL, k)


create dataset
create dataset
feature extraction
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
split grids
spl